In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from bb_setup import setup_2,setup_1
import warnings
from datetime import datetime,timedelta
warnings.filterwarnings("ignore")

In [12]:
#tz = pytz.timezone('America/Sao_Paulo')
ativo = 'WIN@N'
#ativos = ['EURUSD']

pontos = 40
rate_stop = 1
rate_tp = 1.5
spread = 5
var_bb = 2

timeframe = 1

path = r'C:\Program Files\MetaTrader 5\terminal64.exe'

if not mt5.initialize(path = path):
    print("initialize() failed")
    mt5.shutdown()

df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 5000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
#df = df[(df['Data'] > '2022-11-01') & (df['Data'] < '2022-12-30')]

df_ticks = pd.read_csv('./Dados/win_hoje.csv')

df_acao = setup_1(df,df_ticks,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp,timeframe = timeframe)
df_acao = df_acao[(df_acao['Data'] > '2023-01-05')]

In [13]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')


Timeframe: 1 --pontos: 40 --- rate_tp: 1.5 -- var_bb: 2
Pontos_liquido: 120.0 -- Total_operacoes: 24
Agrupado por Hora
  Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0     09              2     1.000000           110.0
1     10              2     0.500000            10.0
2     11              2     0.500000            10.0
3     12              3     0.000000          -135.0
4     13              4     0.750000           120.0
5     14              2     0.500000            10.0
6     15              2     0.500000            10.0
7     16              3     0.666667            65.0
8     17              3     0.333333           -35.0
9     18              1     0.000000           -45.0
Agrupado por Mes
  Mes  qtd_operacoes  taxa_acerto  pontos_liquido
0  01             24          0.5           120.0



In [15]:
df_acao.tail(15)

,Data,Open,Close,acao,stop,tp,resultado_binario,Hora,Hora_h,Mes
4686,2023-01-05 13:12:00,108515.0,108545.0,sell,108585.0,108485.0,0.0,13:12:00,13,01
4703,2023-01-05 13:29:00,108445.0,108420.0,call,108380.0,108480.0,1.0,13:29:00,13,01
4718,2023-01-05 13:44:00,108415.0,108380.0,call,108340.0,108440.0,1.0,13:44:00,13,01
4732,2023-01-05 13:58:00,108325.0,108300.0,call,108260.0,108360.0,1.0,13:58:00,13,01
4753,2023-01-05 14:19:00,108295.0,108230.0,call,108190.0,108290.0,1.0,14:19:00,14,01
4780,2023-01-05 14:46:00,108300.0,108525.0,sell,108565.0,108465.0,0.0,14:46:00,14,01
4814,2023-01-05 15:20:00,108745.0,108870.0,sell,108910.0,108810.0,0.0,15:20:00,15,01
4835,2023-01-05 15:41:00,108880.0,108850.0,call,108810.0,108910.0,1.0,15:41:00,15,01
4871,2023-01-05 16:17:00,108925.0,108945.0,sell,108985.0,108885.0,0.0,16:17:00,16,01
4895,2023-01-05 16:41:00,108935.0,109020.0,sell,109060.0,108960.0,1.0,16:41:00,16,01


In [41]:
lista_media = df_ticks[df_ticks['Data'] >=  '2023-01-03 13:35:00'].head(50)['media']

In [43]:
for valor_atual in lista_media:
    if valor_atual < 106605.0:
        print('certo')
        break
    elif 106755.0	> valor_atual:
        print(valor_atual)
        print('errado')
        break
    else:
        pass

106652.5
errado


In [ ]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes


df_new = pd.DataFrame([[ativo[:-5],var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
            columns = ['ativo','var_bb','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

df_total = pd.concat([df_total,df_new])

print(df_total.to_string())